The previously defined models will have their hyperparameters tuned in order to improve performance. This will be done through the use of Grid Search. While this is not an exhaustive method, it allows for the optimization for the large quantity of models in the analysis. If wanting to re-run this analysis one could choose to use a more exhaustive grid search or an alternative optimization method.

## Read in data / Imports

In [ ]:
import pandas as pd
from numpy import arange
x_train = pd.read_csv('X_train.csv')
y_train = pd.read_csv('y_train.csv')
x_val = pd.read_csv('X_val.csv')
y_val = pd.read_csv('y_val.csv')
model_eval = []

In [ ]:
pip install pygam

In [ ]:
from math import sqrt
from sklearn.model_selection import RepeatedKFold, train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.metrics import r2_score
#Linear Regression
from sklearn.linear_model import LinearRegression
linear_regression_model = LinearRegression()
#Ridge
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
ridge_model = Ridge()
#Lasso
from sklearn import linear_model
lasso_model= linear_model.Lasso()
#Zip
import statsmodels.api as sm
# Note: ZIP requires both endog and exog to be specified when fitting.
# Here we're just initializing the model without fitting it.
zip_model = None
#Trees
from sklearn.tree import DecisionTreeRegressor
dt_model = DecisionTreeRegressor()
#Random Forrest
from sklearn.ensemble import RandomForestRegressor
rf_model = RandomForestRegressor()
#SVM - SVR
from sklearn import svm
SVM = svm.SVR()
#Neural network
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import make_regression
mlp = MLPRegressor()
#Generalized linear model
import statsmodels.api as sm
# Note: GLM requires both endog and exog to be specified when fitting.
# Here we're just initializing the model without fitting it.
glm_model = sm.GLM(endog=y_train, exog=x_train, family=sm.families.Poisson())
#Generalized additive model
from pygam import LinearGAM, s
gam = LinearGAM()

##Linear regression

Has no hyperparameters so cannot be tuned

##Ridge Regression

The alpha hyperparameter will be optimized using gridsearch

In [ ]:
# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# define grid
grid = dict()
grid['alpha'] = arange(0, 1, 0.01)
# define search
search = GridSearchCV(ridge_model, grid, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# perform the search
ridge_results = search.fit(x_train, y_train)
# summarize
print('MAE: %.3f' % ridge_results.best_score_)
print('Config: %s' % ridge_results.best_params_)

MAE: -0.185
Config: {'alpha': 0.99}


The optimum alpha hyperparameter value when using Grid Search cross validation is 0.99. This model is run.

In [ ]:
# Get the best model
best_model = search.best_estimator_

# Evaluate on validation set
y_val_pred = best_model.predict(x_val)

#Metrics
val_mae = mean_absolute_error(y_val, y_val_pred)
val_r2 = r2_score(y_val, y_val_pred)

print(f"Validation set MAE: {val_mae:.3f}")
print(f"Validation set R^2: {val_r2:.3f}")

# Print the best parameters
print('Best hyperparameters:', search.best_params_)

Validation set MAE: 0.156
Validation set R^2: 0.454
Best hyperparameters: {'alpha': 0.99}


This model is added to the new optimized models results dataframe.

In [ ]:
model = 'Ridge'
mae = val_mae
mse = mean_squared_error(y_val, y_val_pred)
rmse = sqrt(mse)
r2 = val_r2
model_eval.append({'Model': model, 'Mean Absolute Error': mae, 'Mean Squared Error': mse, 'RMSE': rmse,'R-squared': r2})

##Lasso

The alpha hyperparameter will be optimized using gridsearch

In [ ]:
# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# define grid
grid = dict()
grid['alpha'] = arange(0, 1, 0.01)
# define search
search = GridSearchCV(lasso_model, grid, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# perform the search
lasso_results = search.fit(x_train, y_train)
# summarize
print('MAE: %.3f' % lasso_results.best_score_)
print('Config: %s' % lasso_results.best_params_)

MAE: -0.162
Config: {'alpha': 0.05}


The optimum alpha hyperparameter value when using Grid Search cross validation is 0.05. This model is run.

In [ ]:
# Get the best model
best_model = search.best_estimator_

# Evaluate on validation set
y_val_pred = best_model.predict(x_val)

#Metrics
val_mae = mean_absolute_error(y_val, y_val_pred)
val_r2 = r2_score(y_val, y_val_pred)

print(f"Validation set MAE: {val_mae:.3f}")
print(f"Validation set R^2: {val_r2:.3f}")

# Print the best parameters
print('Best hyperparameters:', search.best_params_)

Validation set MAE: 0.098
Validation set R^2: 0.642
Best hyperparameters: {'alpha': 0.05}


This model is added to the new optimized models results dataframe.

In [ ]:
model = 'Lasso'
mae = val_mae
mse = mean_squared_error(y_val, y_val_pred)
rmse = sqrt(mse)
r2 = val_r2
model_eval.append({'Model': model, 'Mean Absolute Error': mae, 'Mean Squared Error': mse, 'RMSE': rmse,'R-squared': r2})

## Trees

The maximum depth, minimum samples per split, min samples per leaf and maximum features hyperparameters will be optimized using grid search.

In [ ]:
# Define the parameter grid
param_grid = {
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 5],
    'max_features': ['sqrt', 'log2']
}

# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# define search
search = GridSearchCV(dt_model, param_grid, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# perform the search
tree_results = search.fit(x_train, y_train)
# summarize
print('MAE: %.3f' % tree_results.best_score_)
print('Config: %s' % tree_results.best_params_)

MAE: -0.144
Config: {'max_depth': 30, 'max_features': 'log2', 'min_samples_leaf': 2, 'min_samples_split': 2}


The optimum  hyperparameter values when using Grid Search cross validation are:

**['max_depth': 30, 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 5]**

A model using these hyperparameter values is run.

In [ ]:
# Get the best model
best_model = search.best_estimator_

# Evaluate on validation set
y_val_pred = best_model.predict(x_val)

#Metrics
val_mae = mean_absolute_error(y_val, y_val_pred)
val_r2 = r2_score(y_val, y_val_pred)

print(f"Validation set MAE: {val_mae:.3f}")
print(f"Validation set R^2: {val_r2:.3f}")

# Print the best parameters
print('Best hyperparameters:', search.best_params_)

Validation set MAE: 0.133
Validation set R^2: 0.509
Best hyperparameters: {'max_depth': 30, 'max_features': 'log2', 'min_samples_leaf': 2, 'min_samples_split': 2}


This model is added to the new optimized models results dataframe.

In [ ]:
model = 'DT'
mae = val_mae
mse = mean_squared_error(y_val, y_val_pred)
rmse = sqrt(mse)
r2 = val_r2
model_eval.append({'Model': model, 'Mean Absolute Error': mae, 'Mean Squared Error': mse, 'RMSE': rmse,'R-squared': r2})

## Random Forrests

The number of estimators, maximum features, maximum depth and mximum leaf nodes hyperparameters will be optimized using grid search.

In [ ]:
param_grid = {
    'n_estimators': [25, 50],
    'max_features': ['sqrt', 'log2'],
    'max_depth': [3, 6, 9],
    'max_leaf_nodes': [3, 6],
}
# define model evaluation method
cv = RepeatedKFold(n_splits=5, n_repeats=3, random_state=1)
# define search
search = GridSearchCV(rf_model, param_grid, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# perform the search
rf_results = search.fit(x_train, y_train)
# summarize
print('MAE: %.3f' % rf_results.best_score_)
print('Config: %s' % rf_results.best_params_)

MAE: -0.144
Config: {'max_depth': 9, 'max_features': 'log2', 'max_leaf_nodes': 6, 'n_estimators': 50}


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


The optimum  hyperparameter values when using Grid Search cross validation are:

**{'max_depth': 6, 'max_features': 'log2', 'max_leaf_nodes': 6, 'n_estimators': 25}**

A model using these hyperparameter values is run.

In [ ]:
# Get the best model
best_model = search.best_estimator_

# Evaluate on validation set
y_val_pred = best_model.predict(x_val)

#Metrics
val_mae = mean_absolute_error(y_val, y_val_pred)
val_r2 = r2_score(y_val, y_val_pred)

print(f"Validation set MAE: {val_mae:.3f}")
print(f"Validation set R^2: {val_r2:.3f}")

# Print the best parameters
print('Best hyperparameters:', search.best_params_)

Validation set MAE: 0.170
Validation set R^2: 0.287
Best hyperparameters: {'max_depth': 9, 'max_features': 'log2', 'max_leaf_nodes': 6, 'n_estimators': 50}


This model is added to the new optimized models results dataframe.

In [ ]:
model = 'RF'
mae = val_mae
mse = mean_squared_error(y_val, y_val_pred)
rmse = sqrt(mse)
r2 = val_r2
model_eval.append({'Model': model, 'Mean Absolute Error': mae, 'Mean Squared Error': mse, 'RMSE': rmse,'R-squared': r2})

##SVM

The number of estimators, maximum features, maximum depth and mximum leaf nodes hyperparameters will be optimized using grid search.

In [ ]:
param_grid = { 'C':[0.001, 0.1, 10, 20],'kernel':['rbf','poly','sigmoid','linear'],'gamma':[0.010, 0.015, 0.020, 0.025, 0.03]}
# define model evaluation method
cv = RepeatedKFold(n_splits=5, n_repeats=3, random_state=1)
# define search
search = GridSearchCV(svm.SVR(), param_grid, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
svm_results = search.fit(x_train,y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
# Get the best model
best_model = search.best_estimator_

# Evaluate on validation set
y_val_pred = best_model.predict(x_val)

#Metrics
val_mae = mean_absolute_error(y_val, y_val_pred)
val_r2 = r2_score(y_val, y_val_pred)

print(f"Validation set MAE: {val_mae:.3f}")
print(f"Validation set R^2: {val_r2:.3f}")

# Print the best parameters
print('Best hyperparameters:', search.best_params_)

Validation set MAE: 0.212
Validation set R^2: -0.388
Best hyperparameters: {'C': 20, 'gamma': 0.03, 'kernel': 'poly'}


The optimum hyperparameter values when using Grid Search cross validation are:

**{'C': 20, 'gamma': 0.03, 'kernel': 'poly'}**

A model using these hyperparameter values is run.

In [ ]:
# Get the best model
best_model = search.best_estimator_

# Evaluate on validation set
y_val_pred = best_model.predict(x_val)

#Metrics
val_mae = mean_absolute_error(y_val, y_val_pred)
val_r2 = r2_score(y_val, y_val_pred)

print(f"Validation set MAE: {val_mae:.3f}")
print(f"Validation set R^2: {val_r2:.3f}")

# Print the best parameters
print('Best hyperparameters:', search.best_params_)

Validation set MAE: 0.212
Validation set R^2: -0.388
Best hyperparameters: {'C': 20, 'gamma': 0.03, 'kernel': 'poly'}


This model is added to the new optimized models results dataframe.

In [ ]:
model = 'SVM'
mae = val_mae
mse = mean_squared_error(y_val, y_val_pred)
rmse = sqrt(mse)
r2 = val_r2
model_eval.append({'Model': model, 'Mean Absolute Error': mae, 'Mean Squared Error': mse, 'RMSE': rmse,'R-squared': r2})

## Neural Network


The activation function, solver and alpha hyperparameters will be optimized using grid search.

In [ ]:
# Define the parameter grid
param_grid = {
    'activation': ['tanh', 'relu'],
    'solver': ['adam', 'lbfgs'],
    'alpha': [0.0001, 0.001, 0.01]
}

In [ ]:
# Step 4: Initialize MLPRegressor and GridSearchCV
mlp = MLPRegressor()
grid_search = GridSearchCV(estimator=mlp, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error', n_jobs=-1)

# Step 5: Fit the model
nn_results = grid_search.fit(x_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
# Get the best model
best_model = grid_search.best_estimator_

# Evaluate on validation set
y_val_pred = best_model.predict(x_val)

#Metrics
val_mae = mean_absolute_error(y_val, y_val_pred)
val_r2 = r2_score(y_val, y_val_pred)

print(f"Validation set MAE: {val_mae:.3f}")
print(f"Validation set R^2: {val_r2:.3f}")

# Print the best parameters
print('Best hyperparameters:', grid_search.best_params_)

Validation set MAE: 0.106
Validation set R^2: 0.715
Best hyperparameters: {'activation': 'relu', 'alpha': 0.001, 'solver': 'adam'}


This model is added to the new optimized models results dataframe.

In [ ]:
model = 'NN'
mae = val_mae
mse = mean_squared_error(y_val, y_val_pred)
rmse = sqrt(mse)
r2 = val_r2
model_eval.append({'Model': model, 'Mean Absolute Error': mae, 'Mean Squared Error': mse, 'RMSE': rmse,'R-squared': r2})

##GLM

The family hyperparameter will be optimized. This refers to the distribution of the target feature. Running the models with multiple possible families should allow for the identification of the best family.

In [ ]:
import statsmodels.api as sm

In [ ]:
# Define families to evaluate
families = [sm.families.Poisson(), sm.families.Gaussian(), sm.families.Binomial()]

# Initialize features to store the best results
best_mae, best_mse, best_rmse, best_r2 = float('inf'), float('inf'), float('inf'), float('-inf')
best_family = None

# Iterate over each family
for family in families:
    model = 'GLM'

    # Add intercept term
    X_train_sm = sm.add_constant(x_train)
    X_val_sm = sm.add_constant(x_val)

    # Fit the GLM model
    glm_model = sm.GLM(endog=y_train, exog=X_train_sm, family=family)
    glm_result = glm_model.fit()

    # Predict the values for the validation set
    predictions = glm_result.predict(X_val_sm)

    # Calculate evaluation metrics
    mae = mean_absolute_error(y_val, predictions)
    mse = mean_squared_error(y_val, predictions)
    rmse = sqrt(mse)
    r2 = r2_score(y_val, predictions)

    # Check if this configuration is better
    if mse < best_mse:
        best_mae, best_mse, best_rmse, best_r2 = mae, mse, rmse, r2
        best_family = family

    print(f"Family: {family.__class__.__name__}")
    print(f"Mean Absolute Error: {mae:.2f}")
    print(f"Mean Squared Error: {mse:.2f}")
    print(f"RMSE: {rmse:.2f}")
    print(f"R-squared: {r2:.2f}")
    print("-" * 30)

# Print the best results
print(f'Best Family: {best_family.__class__.__name__}')
print(f'Mean Absolute Error: {best_mae:.2f}')
print(f'Mean Squared Error: {best_mse:.2f}')
print(f'RMSE: {best_rmse:.2f}')
print(f'R-squared: {best_r2:.2f}')

Family: Poisson
Mean Absolute Error: 0.32
Mean Squared Error: 0.16
RMSE: 0.40
R-squared: -2.29
------------------------------
Family: Gaussian
Mean Absolute Error: 0.27
Mean Squared Error: 0.09
RMSE: 0.29
R-squared: -0.80
------------------------------
Family: Binomial
Mean Absolute Error: 0.53
Mean Squared Error: 0.42
RMSE: 0.65
R-squared: -7.77
------------------------------
Best Family: Gaussian
Mean Absolute Error: 0.27
Mean Squared Error: 0.09
RMSE: 0.29
R-squared: -0.80


/usr/local/lib/python3.10/dist-packages/statsmodels/regression/_tools.py:121: RuntimeWarning: divide by zero encountered in scalar divide
  scale = np.dot(wresid, wresid) / df_resid
/usr/local/lib/python3.10/dist-packages/statsmodels/regression/_tools.py:121: RuntimeWarning: divide by zero encountered in scalar divide
  scale = np.dot(wresid, wresid) / df_resid
/usr/local/lib/python3.10/dist-packages/statsmodels/regression/_tools.py:121: RuntimeWarning: divide by zero encountered in scalar divide
  scale = np.dot(wresid, wresid) / df_resid
/usr/local/lib/python3.10/dist-packages/statsmodels/genmod/families/links.py:198: RuntimeWarning: overflow encountered in exp
  t = np.exp(-z)


Best GLM is Gaussian

This model is added to the new optimized models results dataframe.

In [ ]:
model = 'GLM'
mae = val_mae
mse = mean_squared_error(y_val, y_val_pred)
rmse = sqrt(mse)
r2 = val_r2
model_eval.append({'Model': model, 'Mean Absolute Error': best_mae, 'Mean Squared Error': best_mse, 'RMSE': best_rmse,'R-squared': best_r2})

## ZIP

The inflation model parameter will be optimized

In [ ]:
# Ensure the data is in numpy array format
import numpy as np
X_traina = np.array(x_train)
y_traina = np.array(y_train)
X_vala = np.array(x_val)
y_vala = np.array(y_val)

from sklearn.preprocessing import StandardScaler
# Scale the features
scaler = StandardScaler()
X_traina = scaler.fit_transform(X_traina)
X_vala = scaler.transform(X_vala)

# Define the parameter grid for the inflation model
inflation_models = ['logit', 'probit']

# Track the best model and parameters
best_aic = float("inf")
best_params = None
best_model = None

# Manual grid search
for inflation in inflation_models:
     # Initialize and fit the Zero-Inflated Poisson model with the current inflation model
    zip_model = sm.ZeroInflatedPoisson(endog=y_traina, exog=X_traina, exog_infl=X_traina, inflation=inflation)
    zip_result = zip_model.fit(disp=False, maxiter=1000)  # Suppress output during fitting

    # Get the AIC (Akaike Information Criterion) for model selection
    aic = zip_result.aic

    # Track the best model
    if aic < best_aic:
        best_aic = aic
        best_params = {'inflation': inflation}
        best_model = zip_result

# Print the best hyperparameters
print("Best hyperparameters:", best_params)

/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:595: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '
/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:595: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '
/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:595: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '


Best hyperparameters: {'inflation': 'probit'}


/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:595: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '


Best Hyperparams: {'inflation': 'probit'}

In [ ]:
best_model

This model is added to the new optimized models results dataframe.

In [ ]:
model = 'ZIP'
# Predict using the best model on the validation set
y_val_pred = best_model.predict(exog=X_vala, exog_infl=X_vala)
# Calculate MAE, MSE, RMSE, and R-squared
mae = mean_absolute_error(y_vala, y_val_pred)
mse = mean_squared_error(y_vala, y_val_pred)
rmse = sqrt(mse)
val_r2 = r2_score(y_vala, y_val_pred)
r2 = val_r2

print(f"Validation set MAE: {mae:.3f}")
print(f"Validation set R^2: {r2:.3f}")
model_eval.append({'Model': model, 'Mean Absolute Error': mae, 'Mean Squared Error': mse, 'RMSE': rmse,'R-squared': r2})

Validation set MAE: 0.301
Validation set R^2: -1.154


##GAM

The hyperparameters to be optimized are:


1.   n_splines
2.   spline_order


pygam models do not work with Grid Search cross validation.

Set up manual search instead.


In [ ]:
import numpy as np
from pygam import LinearGAM, s
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
n_splines = [5, 10, 20, 30]  # Number of splines
spline_order = [2, 3]       # Order of the splines

# Ensure the data is in numpy array format
x_traina = np.array(x_train)
y_traina = np.array(y_train)
x_vala = np.array(x_val)
y_vala = np.array(y_val)

# Track the best model and parameters
best_mse = float("inf")
best_params = None
best_gam = None

# Manual grid search
for n_spline in n_splines:
  for spline_orders in spline_order:
    # Define the GAM with current hyperparameters
    gam = LinearGAM(s(0, n_splines=n_spline, spline_order=spline_orders))
    # Train the model
    gam.fit(x_traina, y_traina)

    # Predict on validation set
    y_val_pred = gam.predict(x_vala)

    # Calculate mean squared error
    mse = mean_squared_error(y_vala, y_val_pred)

    # Track the best model
    if mse < best_mse:
        best_mse = mse
        best_params = {'n_splines': n_spline, 'spline_order': spline_orders}
        best_gam = gam

# Print the best hyperparameters
print("Best hyperparameters:", best_params)
print("Best model:", best_gam )
# Evaluate the best model on the test set
y_pred = best_gam.predict(x_vala)
mse = np.mean((y_vala - y_pred) ** 2)
print("Validation MSE:", mse)

Best hyperparameters: {'n_splines': 30, 'spline_order': 2}
Best model: LinearGAM(callbacks=[Deviance(), Diffs()], fit_intercept=True, 
   max_iter=100, scale=None, terms=s(0) + intercept, tol=0.0001, 
   verbose=False)
Validation MSE: 0.05268166809427699


Optimum GAM construction: {'n_splines': 30, 'spline_order': 2}

This model is added to the new optimized models results dataframe.

In [ ]:
model = 'GAM'
mae = val_mae
mse = mean_squared_error(y_val, y_val_pred)
rmse = sqrt(mse)
val_r2 = r2_score(y_val, y_val_pred)
r2 = val_r2
model_eval.append({'Model': model, 'Mean Absolute Error': mae, 'Mean Squared Error': mse, 'RMSE': rmse,'R-squared': r2})

#Final Results

In [ ]:
model_eval_df = []

Model results are turned into a pandas dataframe and displayed below.

In [ ]:
model_eval_df = pd.DataFrame(model_eval)
model_eval_df

,Model,Mean Absolute Error,Mean Squared Error,RMSE,R-squared
0,Ridge,0.155577,0.026132,0.161654,0.454050
1,Lasso,0.097783,0.017157,0.130986,0.641547
2,DT,0.132595,0.023503,0.153307,0.508979
3,RF,0.169912,0.034136,0.184758,0.286839
4,SVM,0.212378,0.066452,0.257784,-0.388322
5,NN,0.105600,0.013657,0.116861,0.714689
6,GLM,0.269005,0.086119,0.293460,-0.799186
7,ZIP,0.300898,0.103095,0.321085,-1.153865
8,GAM,0.105600,0.052685,0.229532,-0.100693


In [ ]:
model_names = ['Ridge Regression', 'Lasso Regression',  'Decision Tree', 'Random Forrest', 'SVM', 'Neural Network', 'GLM', 'ZIP', 'GAM']
model_eval_df.insert(1, 'model_nice_names', model_names)

These more user friendly model names are passed to the dataset to allow for easier reading.

In [ ]:
model_eval_df

,Model,model_nice_names,Mean Absolute Error,Mean Squared Error,RMSE,R-squared
0,Ridge,Ridge Regression,0.155577,0.026132,0.161654,0.454050
1,Lasso,Lasso Regression,0.097783,0.017157,0.130986,0.641547
2,DT,Decision Tree,0.132595,0.023503,0.153307,0.508979
3,RF,Random Forrest,0.169912,0.034136,0.184758,0.286839
4,SVM,SVM,0.212378,0.066452,0.257784,-0.388322
5,NN,Neural Network,0.105600,0.013657,0.116861,0.714689
6,GLM,GLM,0.269005,0.086119,0.293460,-0.799186
7,ZIP,ZIP,0.300898,0.103095,0.321085,-1.153865
8,GAM,GAM,0.105600,0.052685,0.229532,-0.100693


In [ ]:
from google.colab import files
model_eval_df.to_csv('Optimized_models_results.csv', index=None)
files.download("Optimized_models_results.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>